In [1]:
"""
Copyright (c) 2019-present NAVER Corp.
MIT License
"""

# !pip install opencv  # ! used for command line commands in notebooks

'\nCopyright (c) 2019-present NAVER Corp.\nMIT License\n'

In [2]:
1# -*- coding: utf-8 -*-
import sys
import os
import time
import argparse

In [3]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

In [4]:
from PIL import Image

In [5]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/CRAFT FYP

/content/drive/MyDrive/Colab Notebooks/CRAFT FYP


In [ ]:
!pip install scikit-image


In [ ]:
import cv2

from skimage import io
import numpy as np
import craft_utils
import imgproc
import file_utils

import json
import zipfile

In [ ]:
print(cudnn.is_available())
print (torch.cuda.is_available())

In [ ]:
!pip uninstall craft



In [ ]:
from craft import CRAFT
#!ls


In [ ]:
from collections import OrderedDict
def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

In [ ]:
def str2bool(v):
    return v.lower() in ("yes", "y", "true", "t", "1")

In [ ]:
parser = argparse.ArgumentParser(description='CRAFT Text Detection')
parser.add_argument('--trained_model', default='weights/craft_mlt_25k.pth', type=str, help='pretrained model')
parser.add_argument('--text_threshold', default=0.7, type=float, help='text confidence threshold') #0.7
parser.add_argument('--low_text', default=0.4, type=float, help='text low-bound score')
parser.add_argument('--link_threshold', default=0.2, type=float, help='link confidence threshold') #0.2
parser.add_argument('--cuda', default=True, type=str2bool, help='Use cuda for inference')
parser.add_argument('--canvas_size', default=1280, type=int, help='image size for inference')
parser.add_argument('--mag_ratio', default=2.0, type=float, help='image magnification ratio') # 1.5
parser.add_argument('--poly', default=False, action='store_true', help='enable polygon type')
parser.add_argument('--show_time', default=False, action='store_true', help='show processing time')
parser.add_argument('--test_folder', default='Data', type=str, help='folder path to input images')
parser.add_argument('--refine', default=False, action='store_true', help='enable link refiner')
parser.add_argument('--refiner_model', default='weights/craft_refiner_CTW1500.pth', type=str, help='pretrained refiner model')

In [ ]:
args, unknown = parser.parse_known_args()

In [ ]:
""" For test images in a folder """
image_list, _, _ = file_utils.get_files(args.test_folder)
print(args.test_folder)

In [ ]:
result_folder = './result/'
if not os.path.isdir(result_folder):
    os.mkdir(result_folder)

In [ ]:
def test_net(net, image, text_threshold, link_threshold, low_text, cuda, poly, refine_net=None):
    t0 = time.time()

    # resize
    img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, args.canvas_size, interpolation=cv2.INTER_LINEAR, mag_ratio=args.mag_ratio)
    ratio_h = ratio_w = 1 / target_ratio

    # preprocessing
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
    x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]
    if cuda:
        x = x.cuda()

    # forward pass
    with torch.no_grad():
        y, feature = net(x)

    # make score and link map
    score_text = y[0,:,:,0].cpu().data.numpy()
    score_link = y[0,:,:,1].cpu().data.numpy()

    # refine link
    if refine_net is not None:
        with torch.no_grad():
            y_refiner = refine_net(y, feature)
        score_link = y_refiner[0,:,:,0].cpu().data.numpy()

    t0 = time.time() - t0
    t1 = time.time()

    # Post-processing
    boxes, polys = craft_utils.getDetBoxes(score_text, score_link, text_threshold, link_threshold, low_text, poly)

    # coordinate adjustment
    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    polys = craft_utils.adjustResultCoordinates(polys, ratio_w, ratio_h)
    for k in range(len(polys)):
        if polys[k] is None: polys[k] = boxes[k]

    t1 = time.time() - t1

    # render results (optional)
    render_img = score_text.copy()
    render_img = np.hstack((render_img, score_link))
    ret_score_text = imgproc.cvt2HeatmapImg(render_img)

    if args.show_time : print("\ninfer/postproc time : {:.3f}/{:.3f}".format(t0, t1))

    return boxes, polys, ret_score_text

In [ ]:

if __name__ == '__main__':
    # load net
    net = CRAFT()     # initialize

    print('Loading weights from checkpoint (' + args.trained_model + ')')
    if args.cuda:
        net.load_state_dict(copyStateDict(torch.load(args.trained_model)))
    else:
        net.load_state_dict(copyStateDict(torch.load(args.trained_model, map_location='cpu')))

    if args.cuda:
        net = net.cuda()
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = False

    net.eval()

    # LinkRefiner
    refine_net = None
    if args.refine:
        from refinenet import RefineNet
        refine_net = RefineNet()
        print('Loading weights of refiner from checkpoint (' + args.refiner_model + ')')
        if args.cuda:
            refine_net.load_state_dict(copyStateDict(torch.load(args.refiner_model)))
            refine_net = refine_net.cuda()
            refine_net = torch.nn.DataParallel(refine_net)
        else:
            refine_net.load_state_dict(copyStateDict(torch.load(args.refiner_model, map_location='cpu')))

        refine_net.eval()
        args.poly = True

    t = time.time()

    # load data
    for k, image_path in enumerate(image_list):
        print("Test image {:d}/{:d}: {:s}".format(k+1, len(image_list), image_path), end='\r')
        image = imgproc.loadImage(image_path)

        bboxes, polys, score_text = test_net(net, image, args.text_threshold, args.link_threshold, args.low_text, args.cuda, args.poly, refine_net)

        # save score text
        filename, file_ext = os.path.splitext(os.path.basename(image_path))
        mask_file = result_folder + "/res_" + filename + '_mask.jpg'
        cv2.imwrite(mask_file, score_text)

        file_utils.saveResult(image_path, image[:,:,::-1], polys, dirname=result_folder)

    print("elapsed time : {}s".format(time.time() - t))

In [ ]:
%cd result

In [ ]:

def group_boxes_by_line(boxes, text_threshold):
    lines = []
    sorted_boxes = sorted(boxes, key=lambda x: (x[1], x[0]))
    current_line = [sorted_boxes[0]]
    for box in sorted_boxes[1:]:
        if box[1] - current_line[-1][1] < text_threshold:
            current_line.append(box)
        else:
            lines.append(current_line)
            current_line = [box]
    lines.append(current_line)
    return lines
def read_bounding_boxes_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        print(lines)
        boxes = [list(map(int, line.strip().split(','))) for line in lines]
    return boxes

# Read word-level bounding boxes from a text file
word_boxes = read_bounding_boxes_from_file('res_Multiple_Qs.txt')

# Group the word-level bounding boxes into line-level bounding boxes
text_threshold = 10  # Adjust this threshold as needed
line_boxes = group_boxes_by_line(word_boxes, 20) # threshold was 50 for the other image

# Now `line_boxes` contains bounding boxes grouped by lines


In [ ]:
print(line_boxes[0])

In [ ]:
%cd ..

In [ ]:
%cd  Data

In [ ]:
from PIL import Image

# Assuming `image_path` is the path to your input image
image_path = 'Multiple_Qs.jpeg'
image = Image.open(image_path)

# Assuming `line_boxes` contains the line-level bounding boxes in the format (x_min, y_min, x_max, y_max)
cropped_images = []
result2=[]
for i, line in enumerate(line_boxes):
  x_min = 9999
  y_min= 9999
  x_max=-9999
  y_max=-9999

  for j, box in enumerate(line):
        # Assuming each box has 4 points
    x_min = min(box[0], box[2], box[4], box[6],x_min)
    y_min = min(box[1], box[3], box[5], box[7],y_min)
    x_max = max(box[0], box[2], box[4], box[6],x_max)
    y_max = max(box[1], box[3], box[5], box[7],y_max)
  result2.append((x_min,y_min,x_max,y_min,x_min,y_max,x_max,y_max))

# Now `cropped_images` contains PIL image objects of the cropped lines.
# You can perform further operations on them.





In [ ]:
display(image)

In [ ]:
img_cropped=[]
for i in range(len(result2)): #len(result2)-1
  x1, y1, x2, y2, x3, y3, x4, y4 = result2[i]
  left = min(x1, x4)
  upper = min(y1, y2)
  right = max(x2, x3)
  lower = max(y3, y4)
  img_cropped.append(image.crop((x1, y1, x4, y4)))




In [ ]:
%cd ..

In [ ]:
%cd result

In [ ]:
%cd Crop

In [ ]:
for i in range (0,len(img_cropped)):
  img_cropped[i].save(f'Test_Image{i}.jpg')



In [ ]:
!pip install -q transformers

In [ ]:
from transformers import VisionEncoderDecoderModel
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

In [ ]:
import re

def remove_non_alphabet_characters(input_string):
    result_string = re.sub(r'[^a-zA-Z\s]', '', input_string)
    return result_string

In [ ]:
  resultant_string =""
  from PIL import Image, ImageFilter
  from transformers import TrOCRProcessor
  for i in range (0,len(img_cropped)):
    image=Image.open(f'Test_Image{i}.jpg')
    sharpened_img = image.filter(ImageFilter.SHARPEN)
    processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
    # calling the processor is equivalent to calling the feature extractor
    pixel_values = processor(sharpened_img, return_tensors="pt").pixel_values

    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_text=remove_non_alphabet_characters(generated_text)
    generated_text=generated_text.strip()
    if (len(generated_text)>2):
      resultant_string+=generated_text.lower()
      resultant_string+=" "
      print(f"{i}:{generated_text}")

In [ ]:
split_result = resultant_string.split("question")
for i in range (1,len(split_result)):
  temp=split_result[i].split("answer")
  print(temp)

[' what were the main causes of the french revolution ', ' the french revolution began in  was fueled by combination of economic social and political factors widespread financial crisis high taries and inequality between the nobility and commoners exacerbated by years of feudalism led to a sense of discontent amongest the masses ']
[' how did the industrial revolution impact society in the th century ', ' the industrial revolution spanning from the late th to the mid th century being about profound charges in society  the argararian economics were shifted industrialized economy and led to the urbanization of people ']


['', ' what were the main causes of the french revolution answer the french revolution began in  was fueled by combination of economic social and political factors widespread financial crisis high taries and inequality between the nobility and commoners exacerbated by years of feudalism led to a sense of discontent amongest the masses ', ' how did the industrial revolution impact society in the th century answer the industrial revolution spanning from the late th to the mid th century being about profound charges in society  the argararian economics were shifted industrialized economy and led to the urbanization of people ']
